Interactive 2D Visualisation of the obtained goal spaces in the IMGEP experiments (video *Visualisation of Learned Goal Spaces* on the website https://automated-discovery.github.io/index.html)

In [7]:
# default print properties
multiplier = 2

pixel_cm_ration = 36.5

width_full = int(13.95 * pixel_cm_ration) * multiplier
width_full_smaller = int((13.95-1) * pixel_cm_ration) * multiplier
width_half = int(13.95/2 * pixel_cm_ration) * multiplier
width_half_smaller = int((13.95-1)/2 * pixel_cm_ration) * multiplier

height_default_1 = int(3.5 * pixel_cm_ration) * multiplier
height_default_4 = int(7 * pixel_cm_ration) * multiplier

# margins in pixel
top_margin = 0 * multiplier 
left_margin = 0 * multiplier 
right_margin = 0 * multiplier 
bottom_margin = 0 * multiplier 

font_size = 10 * multiplier 
font_family='Times New Roman'

line_width = 2 * multiplier 

In [2]:
# Define and load data
import autodisc as ad
import ipywidgets
import plotly
import numpy as np
import collections
plotly.offline.init_notebook_mode(connected=True)

data_filters = collections.OrderedDict()
data_filters['dead'] = ('classifier_dead.data', '==', True)
data_filters['non-animal'] = (('classifier_dead.data', '==', False), 'and', ('classifier_animal.data', '==', False))
data_filters['animal'] = ('classifier_animal.data', '==', True)

org_experiment_definitions = dict()

org_experiment_definitions['main_paper'] = [
    dict(id = '109',
         directory = '../experiments/experiment_000109',
         name = 'IMGEP-HGS',
         is_default = True),
    
    dict(id = '202',
         directory = '../experiments/experiment_000202',
         name = 'IMGEP-PGL',
         is_default = True),

    dict(id = '302',
         directory = '../experiments/experiment_000302',
         name = 'IMGEP-OGL',
         is_default = True),
]

repetition_ids = list(range(1))

# define names and load the data
experiment_name_format = '<name>' # <id>, <name>

# global experiment_definitions
experiment_definitions = []
experiment_statistics = []

current_experiment_list = 'main_paper'

experiment_definitions = []
for org_exp_def in org_experiment_definitions[current_experiment_list]:
    new_exp_def = dict()
    new_exp_def['directory'] = org_exp_def['directory']
    if 'is_default' in org_exp_def:
        new_exp_def['is_default'] = org_exp_def['is_default']

    if 'name' in org_exp_def:
        new_exp_def['id'] = ad.gui.jupyter.misc.replace_str_from_dict(experiment_name_format, {'id': org_exp_def['id'], 'name': org_exp_def['name']})
    else:
        new_exp_def['id'] = ad.gui.jupyter.misc.replace_str_from_dict(experiment_name_format, {'id': org_exp_def['id']})

    experiment_definitions.append(new_exp_def)

experiment_statistics = dict()
for experiment_definition in experiment_definitions:
    experiment_statistics[experiment_definition['id']] = ad.gui.jupyter.misc.load_statistics(experiment_definition['directory'])
       

In [3]:
# Compute PCA + TSNE + UMAP per experiment repetition
import os
import sklearn

def do_dimension_reduction(algorithm):

    data = dict()
    
    experiment_ids = [exp_def['id'] for exp_def in experiment_definitions]
    for experiment_id in experiment_ids: 
        experiment_idx = experiment_ids.index(experiment_id)
        data[experiment_idx] = dict()
        
        for repetition_idx in repetition_ids:
            experiment_statistics[experiment_id][repetition_idx] = ad.gui.jupyter.misc.load_statistics(os.path.join(experiment_definitions[experiment_idx]['directory'], 'repetition_{:06d}'.format(repetition_idx)))        
        for repetition_idx in repetition_ids:
            try:
                data_in_goal_space = experiment_statistics[experiment_id][repetition_idx]['representations']['coordinates_in_goal_space']
                data_in_goal_space = np.nan_to_num(data_in_goal_space)
                data_in_goal_space = sklearn.preprocessing.MinMaxScaler().fit_transform(data_in_goal_space)

                # compute 
                fitted_data = algorithm.fit_transform(data_in_goal_space)
                fitted_data = sklearn.preprocessing.MinMaxScaler().fit_transform(fitted_data)            
                data[experiment_idx][repetition_idx] = fitted_data

            except Exception as e:
                raise RuntimeError('Error for experiment {!r}, repetition {!r}!'.format(experiment_id, repetition_idx) ) from e
        
    return data


## PCA
import sklearn.decomposition
pca_algorithm = sklearn.decomposition.PCA(n_components=2)
pca_data = do_dimension_reduction(pca_algorithm)

## TSNE
import sklearn.manifold
tsne_algorithm = sklearn.manifold.TSNE(n_components=2, 
                                       init='random', 
                                       random_state=0, 
                                       perplexity=50)
tsne_data = do_dimension_reduction(tsne_algorithm)

## UMAP
from umap import UMAP
umap_algorithm = UMAP(n_neighbors=10, 
                       min_dist=0.1, 
                       metric='euclidean')
umap_data = do_dimension_reduction(umap_algorithm)

/home/mayalen/anaconda3/envs/autodisc/lib/python3.6/site-packages/umap/spectral.py:229: UserWarning:

Embedding a total of 2 separate connected components using meta-embedding (experimental)



In [8]:
 # general layout of figure
plot_layout = dict(
        xaxis=dict(
            range=[-0.03,1.03],
            showgrid=True,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False
        ),
        yaxis=dict(
            range=[-0.03,1.03],
            showgrid=True,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False
        ),
        font = dict(
            family=font_family, 
            size=font_size, 
            ),
        width=width_full_smaller, # in cm
        height=height_default_4, # in cm
        
        margin = dict(
            l=left_margin, #left margin in pixel
            r=right_margin, #right margin in pixel
            b=bottom_margin, #bottom margin in pixel
            t=top_margin,  #top margin in pixel
            ),        
        
        showlegend=True,
        legend=dict(
            xanchor='right',
            yanchor='bottom',
            y=0,
            x=1,
            ), 
    )

# Interface

The user can
* Select the experiment and repetition to visualize (press the **Load** button) 
* Select the visualisation dimensionality reduction technique (**PCA, T-SNE, UMAP**)
* **Zoom** in/out and **Pan** in the goal space through plotly toolbar (top-right corner of the figure)
* Select points from the plots (**Box Select** in the tool bar) and visualize the corresponding patterns found by the algorithm

In [10]:
# Launch GUI
from autodisc.gui.jupyter import PlotRepresentationSpaceWidget
gui_config = PlotRepresentationSpaceWidget.get_default_gui_config()
gui_config.latent_space_viewer_widget.image_list_widget.box_layout.height = '500px'
gui_config.latent_space_viewer_widget.visualisation_type_selector.options = ['PCA', 'T-SNE', 'UMAP']
gui_config.latent_space_viewer_widget.visualisation_type_selector.value = 'T-SNE'
gui_config.latent_space_viewer_widget.canvas.layout = plot_layout
PlotRepresentationSpaceWidget(experiment_definitions, repetition_ids, pca_data, tsne_data, umap_data, experiment_statistics, data_filters, config=gui_config)


PlotRepresentationSpaceWidget(children=(ExperimentRepetitionLoaderWidget(children=(Dropdown(description='Exper…